# Ensemble Algorithms

## Intro

Bagging is an ensemble algorithm that fits multiple models on different subsets of a trainingdataset, then combines the predictions from all models.  Random forests are an extension ofbagging that also randomly selects subsets of features used in each data sample.  Both baggingand  random  forests  have  proven  effective  on  a  wide  range  of  different  predictive  modelingproblems.Although  effective,  they  are  not  suited  to  classification  problems  with  a  skewed  classdistribution.  Nevertheless, many modifications to the algorithms have been proposed that adapttheir behavior and make them better suited to a severe class imbalance.  In this tutorial, you willdiscover how to use bagging and random forest for imbalanced classification.  

This tutorial is divided into three parts; they are:

1. Bagging for Imbalanced Classification
2. Random Forest for Imbalanced Classification
3. Easy Ensemble for Imbalanced Classification

## Bagging for Imbalanced Classification
Bootstrap Aggregation, or Bagging for short, is an ensemble machine learning algorithm.  Itinvolves first selecting random samples from the training dataset with replacement, meaningthat a given sample may contain zero, one, or more than one copy of examples in the trainingdataset.  This is called a bootstrap sample.  One weak learner model is then fit on each datasample.  Typically, decision tree models that do not use pruning (e.g.  may overfit their trainingset slightly) are used as weak learners.  Finally, the predictions from all of the fit weak learnersare combined to make a single prediction (e.g.  aggregated).

The process of creating new bootstrap samples and fitting and adding trees to the samplecan continue until no further improvement is seen in the ensemble’s performance on a validationdataset.  This simple procedure often results in better performance than a single well-configureddecision tree algorithm.  Bagging as-is will create bootstrap samples that will not consider theskewed class distribution for imbalanced classification datasets.  As such, although the techniqueperforms well in general, it may not perform well if a severe class imbalance is present.

###  Standard Bagging

Firts let’s evaluate a standard bagged decision treeensemble and use it as a point of comparison.  We can use theBaggingClassifierscikit-sklearnclass to create a bagged decision tree model.  



In [8]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.ensemble  import BaggingClassifier

X, y = make_classification(n_samples = 10_000, n_features=2, n_redundant=0,
                        n_clusters_per_class=1, weights=[.99], flip_y=0, random_state=4)
model = BaggingClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# eval model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.3f" % (np.mean(scores)))

Mean ROC AUC: 0.869


### Bagging With Random Undersampling

There are many ways to adapt bagging for use with imbalanced classification.  Perhaps the most straightforward approach is to apply data sampling on the bootstrap sample prior to fitting the weak learner model.  This might involve oversampling the minority class or undersampling the majority class.

The imbalanced-learn library provides an implementation of UnderBagging.  Specifically, it provides a version of bagging that uses a random undersampling strategy on the majority class within a bootstrap sample in order to balance the two classes.  This is provided in the `BalancedBaggingClassifier` class.


In [9]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

#new model
model = BalancedBaggingClassifier(base_estimator=None)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % (np.mean(scores)))

Mean ROC AUC: 0.965


In  this  case,  we  can  see  a  lift  on  mean  ROC  AUC  from  about  0.87  without  any  datasampling, to about 0.96 with random undersampling of the majority class.  This is not a trueapples-to-apples comparison as we are using the same algorithm implementation from twodifferent libraries, but it makes the general point that balancing the bootstrap prior to fitting aweak learner offers some benefit when the class distribution is skewed.
 
Although the BalancedBaggingClassifierclass uses a decision tree, you can test different models, such as k-nearest neighbors and more.  You can set thebaseestimatorargument whendefining the class to use a different weak learner classifier model.

## Random Forest for Imbalanced Classification 

Random forest is another ensemble of decision tree models and may be considered an improvement upon bagging.  Like bagging, random forest involves selecting bootstrap samples from the training dataset and fitting a decision tree on each.  The main difference is that all features (variables or columns) are not used; instead, a small, randomly selected subset of features (columns) is chosen for each bootstrap sample.  This has the effect of de-correlating the decision trees (making them more independent), and in turn, improving the ensemble prediction.

Again,  random  forest  is  very  effective  on  a  wide  range  of  problems,  but  like  bagging,performance of the standard algorithm is not great on imbalanced classification problems.

### Standard Random Forest

In [10]:
from sklearn.ensemble  import RandomForestClassifier

X, y = make_classification(n_samples = 10_000, n_features=2, n_redundant=0,
                        n_clusters_per_class=1, weights=[.99], flip_y=0, random_state=4)
model = RandomForestClassifier(n_estimators=10)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# eval model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.3f" % (np.mean(scores)))

Mean ROC AUC: 0.882


### Random Forest with class weighting

A simple technique for modifying a decision tree for imbalanced classification is to change the weight that each class has when calculating the impurity score of a chosen split point.  Impurity measures how mixed the groups of samples are for a given split in the training dataset and is typically measured with Gini or entropy.  The calculation can be biased so that a mixture in favor of the minority class is favored, allowing some false positives for the majority class.  This modification of random forest is referred to as Weighted Random Forest.

 The argument value of ‘balanced’ can be provided to class weight to automatically use the inverse weighting from the training dataset, giving focus to the minority class.

In [11]:
model = RandomForestClassifier(n_estimators=10, class_weight='balanced')
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# eval model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.3f" % (np.mean(scores)))

Mean ROC AUC: 0.873


### Random Forest with Bootstrap class weighting

Given that each decision tree is constructed from a bootstrap sample (e.g.  random selection with replacement), the class distribution in the data sample will be different for each tree.  As such, it might be interesting to change the class weighting based on the class distribution in each bootstrap sample, instead of the entire training dataset.  This can be achieved by setting the classweight argument to the value ‘balanced_subsample’. 

In [12]:
model = RandomForestClassifier(n_estimators=10, class_weight='balanced_subsample')
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# eval model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.3f" % (np.mean(scores)))

Mean ROC AUC: 0.872


### Random Forest With Random Undersampling

Another useful modification to random forest is to perform data sampling on the bootstrap sample in order to explicitly change the class distribution.  The BalancedRandomForestClassifier class from the imbalanced-learn library implements this and performs random undersampling of the majority class in each bootstrap sample.  This is generally referred to as Balanced RandomForest.

In [13]:
from imblearn.ensemble import BalancedRandomForestClassifier

# define model
model = BalancedRandomForestClassifier(n_estimators=10)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', n_jobs=-1)
print("Mean ROC AUC: %.3f" % (np.mean(scores)))

Mean ROC AUC: 0.973


### Easy Ensemble

The Easy Ensemble involves creating balanced samples of the training dataset by selecting all examples from the minority class and a subset from the majority class.  Rather than using pruned decision trees, boosted decision trees are used on each subset, specifically the AdaBoost algorithm.  AdaBoost works by first fitting a decision tree on the dataset, then determining the errors made by the tree and weighing the examples in the dataset by those errors so that more attention is paid to the misclassified examples and less to the correctly classified examples.  A subsequent tree is then fit on the weighted dataset intended to correct the errors.  The process is then repeated for a given number of decision trees.

The `EasyEnsembleClassifier` class from the imbalanced-learn library provides an implementation of the easy ensemble technique

In [14]:
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(n_estimators=10)
cv = RepeatedStratifiedKFold(n_repeats=3, n_splits=10, random_state=1)
# eval model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize the scores
print("Mean ROC AUC: %.3f" % (np.mean(scores)))

Mean ROC AUC: 0.963
